# Analysis of submissions to the Request for Information (RFI) sent out by the Department of Commerce for best practices for disseminating AI-ready data
##### The RFI closed on July 16th

#### Uncomment the cell below and install a package to read pdfs, and import the necessary packages

In [2]:
#!pip install pdftotext

In [31]:
import pdftotext
import os
import pandas as pd
import numpy as np
import re

#### Make a dataframe of the RFI submissions

In [32]:
# truncate submission text
pd.set_option('display.max_colwidth', 50)

RFIs = []
directory = r'RFI_Submissions' # the folder where the pdf submissions are stored locally
for filename in os.listdir(directory):
    if filename != '.DS_Store':
        with open(os.path.join(directory, filename), 'rb') as f:
            filetext = ''
            pdf = pdftotext.PDF(f)
            # pdftotext reads pdfs one page at a time. Concatenate all the pages of a file together
            for page in pdf:
                filetext += page.lower() # save in lowercase to handle different captitalization styles when searching later
        filetext = filetext.replace('\n', ' ')
        RFIs.append([filename,filetext])
RFIs = pd.DataFrame(RFIs).rename({0:'file', 1:'text'}, axis = 1)
RFIs

,file,text
0,019_Esri_RFI.pdf,"july 16, 2024 esri and geospatial artificial ..."
1,030_IBM_RFI.pdf,international business machines (ibm) corporat...
2,103_FAS_RFI.pdf,"oliver wise, chief data officer, department of..."
3,013_Data.World_RFI.pdf,"℅ capital factory 701 brazos, ste 519 austin, ..."
4,009_MySidewalk_RFI.pdf,rfi reponse keep it simple we’ve learned our e...
5,014_Elsevier_RFI.pdf,ai and open government data assets request for...
6,100_Virginia_RFI.pdf,"roya pakzad founder and director, taraaz data ..."
7,038_Replica_RFI.pdf,"july 16, 2024 dear mr. wise, as a long-time be..."
8,008_Zois_American University.pdf,response to commerce’s rfi on improving data c...
9,020_XBRL_RFI.pdf,"july 16, 2024 victoria houed, ousea u.s. depa..."


In [33]:
# make indexes match the file numbers for easier referencing
idxs = RFIs.file.map(lambda x: x[:3])
RFIs = RFIs.set_index(idxs)
RFIs

,file,text
file,,
019,019_Esri_RFI.pdf,"july 16, 2024 esri and geospatial artificial ..."
030,030_IBM_RFI.pdf,international business machines (ibm) corporat...
103,103_FAS_RFI.pdf,"oliver wise, chief data officer, department of..."
013,013_Data.World_RFI.pdf,"℅ capital factory 701 brazos, ste 519 austin, ..."
009,009_MySidewalk_RFI.pdf,rfi reponse keep it simple we’ve learned our e...
014,014_Elsevier_RFI.pdf,ai and open government data assets request for...
100,100_Virginia_RFI.pdf,"roya pakzad founder and director, taraaz data ..."
038,038_Replica_RFI.pdf,"july 16, 2024 dear mr. wise, as a long-time be..."
008,008_Zois_American University.pdf,response to commerce’s rfi on improving data c...


#### Find all submissions that mention common recommendations
The 'recs' list of recommendations is based on entries in the external 'Guidelines Appendix Tables' spreadsheet: https://docs.google.com/spreadsheets/d/1AP5s1GENEfR4U2v1_NmAQBcoDOIVADaQxxUD_zfmv94/edit?usp=sharing

In [34]:
#view full values
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

# Maybe implement later: return index out of total text length for relative location in document
recs = [
    ### Technical
    ## Data and Metadata
    ['apache atlas'],['croissant'],['csv'],['dcat '],['dcat-us'],['dublin core'],['hdf5'],
    ['iso 19115','iso19115', '19115'],['json'],[' owl'],['parquet'],['rdf'],['schema.org'],['sparql'],
    ['xml'], ['zip '],
    ## Licensing
    ['apache license', 'apache 2.0'],#'apache' could catch more cases but would include 'apache parquet' which is different
    ['cc-by','cc by', 'ccby'],['cc0', 'cc 0', 'cc-0'],['creative commons'],
    ['mit license', ' mit '], #add other open source licenses?
    ['responsible ai  license'], #RAIL acroym would catch too many false positives
    ## Storage and Publishing
    ['api gateway'],['breadcrumb'],['graphql'],['odrl','open digital rights language'],['openapi'],['restful','rest api'],
    ['robots.txt'],['sitemap'],['streaming api'],['wcag','web content accessibility guidelines'],['webhook'],
    ## Validation and Quality
    ['apache data cleaner'],['ai fairness 360'],['maig','manifesto for ai gov'],['runtime application self-protection',' rasp '],
    ['talend'],

    ### High level  (not alphabetized/reorganized yet)
    ## Data and Metadata
    ['unknown'],#work on
    ['data dictionary'],['readiness level'], #work on
    ['ontolog','controlled vocabular'],['open data format'],['quality metrics embedded in metadata'], #work on
    ['semantic web'],['standardized metadata','metadata standards'],['version control', 'historical version','version history'],
    ## Documentation
    ['api document'],['provenance'],['bias'],#work on
    ['collection method'],['terms of use','usage guideline'],['tutorial','sample code'],
    ## Feedback
    ['consortia'], #work on
    ['hackathon'],['community engage','forum'],['usage tracking'], #add more tracking synonyms
    ## Licensing
    ## Storage and Publishing
    ['many formats','multiple formats'], ['ords','open data rights statement'],['language'], #work on
    ['bulk download'],['standardized url','standard url','consistent url','url consistency','url standard'],
    ['low-bandwidth','low bandwidth'],#work on low internet access
    ['synthetic data'],['data preview'],['portal for ai-ready'], #work on this
    ['plugin', 'plug-in'], #work on this
    ## Validation and Quality
    ['confidence interval','uncertainty measure','measure uncertainty'],['edge case','stress test'],['interoperab'],
    ['linked data standard','linked data principle'],['zero trust','never trust, always verify'],
    ['quality metric','measure quality'],
    
    ## Out of Guidelines Scope
    ['blockchain'],['discriminat'],['ords','open data rights statement'],['odrl','open digital rights language'],
    ['section 508'],['accessibility standard'],
    ['benchmark data','benchmark'], #benchmark gives 11 more, investigate
    ['devsecops'],
]

entries={}
for rec in recs:
    entries[str(rec)]=[]
    for synonym in rec:
        for file,text in zip(RFIs['file'],RFIs['text']):
            entry = [rec, file, re.findall(f'{synonym}', text)]
            if len(entry[2])>0: # if at least one instance of the synonym was found in the file, save it to the dictionary
                if file not in entries[str(rec)]:
                    entries[str(rec)]+= [file]

rec_column = list(entries.keys())
file_column = list(entries.values())
frequent_recs_df = pd.DataFrame({'recommendation':rec_column, 'files':file_column, 
                                 'files_count':map(lambda x: len(x), file_column)})
frequent_recs_df

,recommendation,files,files_count
0,['apache atlas'],[011_WhirlwindTechnologies_RFI.pdf],1
1,['croissant'],"[102_MIT_RFI.pdf, 018_MLCommons_RFI.pdf, 104_Google_RFI.pdf]",3
2,['csv'],"[013_Data.World_RFI.pdf, 009_MySidewalk_RFI.pdf, 014_Elsevier_RFI.pdf, 008_Zois_American University.pdf, 020_XBRL_RFI.pdf, 040_Gretel_RFI.pdf, 017_SAS_RFI.pdf, 015_Element84_RFI.pdf, 033_PROTESORO_RFI.pdf, 023_Deloitte_RFI.pdf, 010_ScaleAI_RFI.pdf, 037_UnstructuredTech_RFI.pdf, 011_WhirlwindTechnologies_RFI.pdf, 034_IEEE_RFI.pdf, 036_Microsoft_RFI.pdf, 027_CSET_RFI.pdf, 028_Kitware_RFI.pdf]",17
3,['dcat '],"[008_Zois_American University.pdf, 034_IEEE_RFI.pdf, 021_Jess_UVirginia_RFI .pdf, 028_Kitware_RFI.pdf]",4
4,['dcat-us'],[011_WhirlwindTechnologies_RFI.pdf],1
5,['dublin core'],"[013_Data.World_RFI.pdf, 014_Elsevier_RFI.pdf, 011_WhirlwindTechnologies_RFI.pdf, 034_IEEE_RFI.pdf, 025_StardogUnion_RFI.pdf]",5
6,['hdf5'],"[013_Data.World_RFI.pdf, 011_WhirlwindTechnologies_RFI.pdf, 028_Kitware_RFI.pdf]",3
7,"['iso 19115', 'iso19115', '19115']","[013_Data.World_RFI.pdf, 011_WhirlwindTechnologies_RFI.pdf]",2
8,['json'],"[013_Data.World_RFI.pdf, 014_Elsevier_RFI.pdf, 008_Zois_American University.pdf, 020_XBRL_RFI.pdf, 040_Gretel_RFI.pdf, 017_SAS_RFI.pdf, 015_Element84_RFI.pdf, 033_PROTESORO_RFI.pdf, 023_Deloitte_RFI.pdf, 010_ScaleAI_RFI.pdf, 037_UnstructuredTech_RFI.pdf, 011_WhirlwindTechnologies_RFI.pdf, 034_IEEE_RFI.pdf, 102_MIT_RFI.pdf, 036_Microsoft_RFI.pdf, 029_Leidos_RFI.pdf, 027_CSET_RFI.pdf, 028_Kitware_RFI.pdf]",18
9,[' owl'],"[011_WhirlwindTechnologies_RFI.pdf, 025_StardogUnion_RFI.pdf]",2


#### Find all licenses mentioned, and the text around them to provide context, in order to find new types that haven't been included yet 
##### This method can be used for any other words for more context and checking the intended meaning


In [35]:
licenses = []
for file,text in zip(RFIs['file'],RFIs['text']): 
    licenses.append([file, re.findall('.{100}license.{150}', text)])
licenses

[['019_Esri_RFI.pdf', []],
 ['030_IBM_RFI.pdf', []],
 ['103_FAS_RFI.pdf', []],
 ['013_Data.World_RFI.pdf',
  ['content licensing originating in the open source community. whether these are specifically software licenses (eg. gpl, cddl, apache), or broader copyright licenses (eg. creative commons, copyleft), a long history of application and effectiveness can be leve',
   'commerce consider to support broad, equitable, and open access to its datasets and metadata? ○ open licenses: adopt open licenses like creative commons cc0 or cc by, which facilitate broad reuse and redistribution with minimal restrictions. if more control ',
   ' attributions. ○ standardized licensing metadata: implement standardized metadata fields to capture license information, ensuring it is consistently conveyed across all datasets. ○ equitable access: ensure no barriers to access by providing data in multiple',
   ' processing methods to enhance trust and reliability? transparency in data sourcing: ● ●  open da

#### Additional Analyses (not initially useful)

In [18]:
### tfidf analysis
# def tfidf_data(reviews_ser, review):
#     # Remove non-alphanumeric characters and split by whitespace
#     words = re.findall(r'\b\w+\b', review)
#     out = pd.DataFrame(pd.Series(words).value_counts(), columns=['count']) 
#     #out = pd.DataFrame(pd.Series(review.split()).value_counts())

#     out['indx'] = out.index
#     out['tf'] = out['count']/len(review.split())
#     out['idf'] = np.log(len(reviews_ser)/ (out['indx'].apply(lambda word: sum(reviews_ser.str.contains(fr'\b{re.escape(word)}\b', regex=True)))))
#     out['tfidf'] = out['tf']*out['idf']
#     return out.drop(columns = 'indx').sort_values(['tfidf', 'count'])

# for filename in os.listdir(directory):
#     if filename!= '.DS_Store' and filename!= 'combined.pdf':
#         with open(os.path.join(directory, filename), 'rb') as f:
#             rfi = ''
#             pdf = pdftotext.PDF(f)
#             for page in pdf:
#                 rfi+=page.lower()
#         print(filename)
#         display(tfidf_data(RFIs, rfi))
#     break


### count all word frequencies overall
# directory = r'RFI_Submissions'
# with open(os.path.join(directory, 'combined.pdf'), 'rb') as f:
#     combined = ''
#     pdf = pdftotext.PDF(f)
#     for page in pdf:
#         combined+=page.lower()
# words = re.findall(r'\b\w+\b', combined)
# all_counts = pd.DataFrame(pd.Series(words).value_counts(), columns=['count']) 
# all_counts